In [1]:
import pandas as pd

# Load the datasets
applications_df = pd.read_csv('cleaned_applications.csv')
branches_df = pd.read_csv('cleaned_branches.csv')
customers_df = pd.read_csv('cleaned_customers.csv')
defaults_df = pd.read_csv('cleaned_defaults.csv')
loans_df = pd.read_csv('cleaned_loans.csv')
transactions_df = pd.read_csv('cleaned_transactions.csv')

print("Datasets loaded successfully.")

Datasets loaded successfully.


In [2]:
# Filter transactions for penalty payments
penalty_transactions_df = transactions_df[transactions_df['Payment_Type'] == 'Penalty']

# Group by date and sum the penalty amounts
penalty_trends_df = penalty_transactions_df.groupby('Transaction_Date')['Amount'].sum().reset_index()

# Convert Transaction_Date to datetime objects
penalty_trends_df['Transaction_Date'] = pd.to_datetime(penalty_trends_df['Transaction_Date'])

# Sort by date
penalty_trends_df = penalty_trends_df.sort_values(by='Transaction_Date')

# Calculate and print total and average overdue amounts from loans data
total_overdue_amount = loans_df['Overdue_Amount'].sum()
average_overdue_amount = loans_df['Overdue_Amount'].mean()

print(f"Total Overdue Amount: {total_overdue_amount}")
print(f"Average Overdue Amount: {average_overdue_amount}")

display(penalty_trends_df.head())

Total Overdue Amount: 1147125736
Average Overdue Amount: 12745.841511111112


Transaction_Date   Amount
0       2019-12-28  3853422
1       2019-12-29  2977316
2       2019-12-30  3345141
3       2019-12-31  3127746
4       2020-01-01  3410491

In [3]:
loan_default_merge_df = pd.merge(defaults_df, loans_df, on='Loan_ID', how='right')

loan_default_merge_df['Default_Amount'] = loan_default_merge_df['Default_Amount'].fillna(0)
loan_default_merge_df['Recovery_Amount'] = loan_default_merge_df['Recovery_Amount'].fillna(0)

loan_default_merge_df['Recovery_Rate'] = loan_default_merge_df.apply(
    lambda row: row['Recovery_Amount'] / row['Default_Amount'] if row['Default_Amount'] > 0 else 0,
    axis=1
)

recovery_by_reason_action = loan_default_merge_df.groupby(['Default_Reason', 'Legal_Action'])['Recovery_Rate'].mean().reset_index()
display(recovery_by_reason_action)

Default_Reason Legal_Action  Recovery_Rate
0   Business Failure           No       0.404936
1   Business Failure          Yes       0.402335
2           Job Loss           No       0.393805
3           Job Loss          Yes       0.418474
4  Medical Emergency           No       0.420908
5  Medical Emergency          Yes       0.414707
6              Other           No       0.386740
7              Other          Yes       0.394871

In [4]:
# 1. Merge loans_df with applications_df
loan_application_df = pd.merge(loans_df, applications_df[['Loan_ID', 'Customer_ID']], on='Loan_ID', how='left')

# Rename the Customer_ID column from applications_df to Customer_ID for consistent merging
loan_application_df.rename(columns={'Customer_ID_y': 'Customer_ID'}, inplace=True)

# 2. Merge with customers_df using the correct Customer_ID
loan_customer_branch_df = pd.merge(loan_application_df, customers_df[['Customer_ID', 'Relationship_Manager_ID', 'Region']], on='Customer_ID', how='left')

# 3. Merge with defaults_df
loan_default_branch_df = pd.merge(loan_customer_branch_df, defaults_df[['Loan_ID', 'Default_Amount', 'Recovery_Amount']], on='Loan_ID', how='left')

# 4. Fill NaN values
loan_default_branch_df['Default_Amount'] = loan_default_branch_df['Default_Amount'].fillna(0)
loan_default_branch_df['Recovery_Amount'] = loan_default_branch_df['Recovery_Amount'].fillna(0)

# 5. Calculate Recovery_Rate
loan_default_branch_df['Recovery_Rate'] = loan_default_branch_df.apply(
    lambda row: row['Recovery_Amount'] / row['Default_Amount'] if row['Default_Amount'] > 0 else 0,
    axis=1
)

# 6. Group by Region and calculate mean Recovery_Rate
regional_recovery_rates = loan_default_branch_df.groupby('Region')['Recovery_Rate'].mean().reset_index()

# 7. Group by Relationship_Manager_ID and calculate mean Recovery_Rate
branch_recovery_rates = loan_default_branch_df.groupby('Relationship_Manager_ID')['Recovery_Rate'].mean().reset_index()

# 8. Display the DataFrames
display(regional_recovery_rates)
display(branch_recovery_rates)

Region  Recovery_Rate
0    Central       0.044880
1       East       0.036069
2      North       0.038026
3  Northeast       0.040617
4      South       0.040617
5       West       0.037337

Relationship_Manager_ID  Recovery_Rate
0                    RM0001       0.026855
1                    RM0002       0.068475
2                    RM0003       0.041400
3                    RM0004       0.079029
4                    RM0005       0.024527
..                      ...            ...
495                  RM0496       0.031713
496                  RM0497       0.040954
497                  RM0498       0.035786
498                  RM0499       0.094641
499                  RM0500       0.031891

[500 rows x 2 columns]

In [5]:
# 1. Summarize penalty payments and overdue trends
print("Summary of Penalty Payments and Overdue Trends:")
print(f"- Total Overdue Amount across all loans: {total_overdue_amount:,.2f}")
print(f"- Average Overdue Amount per loan: {average_overdue_amount:,.2f}")
print("\nPenalty Payment Trends (first 5 dates):")
display(penalty_trends_df.head())
print("\nObservations on Penalty Payments:")
print("The daily total penalty amounts fluctuate over time, indicating variations in late payments.")


# 2. Summarize recovery rates by default reason and legal action
print("\nSummary of Recovery Rates by Default Reason and Legal Action:")
display(recovery_by_reason_action)
print("\nObservations on Recovery Rates by Default Reason and Legal Action:")
print("Recovery rates show some variation based on the default reason and whether legal action was taken.")
print("For instance, 'Job Loss' with Legal Action 'Yes' has a slightly higher average recovery rate than 'Job Loss' with 'No'.")
print("However, the differences appear relatively small, suggesting that while these factors play a role, their impact on the average recovery rate might not be substantial.")

# 3. Summarize recovery rates across regions and branches
print("\nSummary of Recovery Rates Across Regions:")
display(regional_recovery_rates)
print("\nSummary of Recovery Rates Across Branches (Relationship Managers):")
display(branch_recovery_rates.head()) # Displaying head as there are many branches
print("\nObservations on Recovery Rates Across Regions and Branches:")
print("Regional recovery rates show some variation, with 'Central' having the highest average recovery rate and 'East' the lowest among the displayed regions.")
print("Recovery rates also vary significantly across different Relationship Managers, indicating potential differences in collection effectiveness at the branch level (as Relationship Managers are proxies for branches).")

Summary of Penalty Payments and Overdue Trends:
- Total Overdue Amount across all loans: 1,147,125,736.00
- Average Overdue Amount per loan: 12,745.84

Penalty Payment Trends (first 5 dates):


Transaction_Date   Amount
0       2019-12-28  3853422
1       2019-12-29  2977316
2       2019-12-30  3345141
3       2019-12-31  3127746
4       2020-01-01  3410491


Observations on Penalty Payments:
The daily total penalty amounts fluctuate over time, indicating variations in late payments.

Summary of Recovery Rates by Default Reason and Legal Action:


Default_Reason Legal_Action  Recovery_Rate
0   Business Failure           No       0.404936
1   Business Failure          Yes       0.402335
2           Job Loss           No       0.393805
3           Job Loss          Yes       0.418474
4  Medical Emergency           No       0.420908
5  Medical Emergency          Yes       0.414707
6              Other           No       0.386740
7              Other          Yes       0.394871


Observations on Recovery Rates by Default Reason and Legal Action:
Recovery rates show some variation based on the default reason and whether legal action was taken.
For instance, 'Job Loss' with Legal Action 'Yes' has a slightly higher average recovery rate than 'Job Loss' with 'No'.
However, the differences appear relatively small, suggesting that while these factors play a role, their impact on the average recovery rate might not be substantial.

Summary of Recovery Rates Across Regions:


Region  Recovery_Rate
0    Central       0.044880
1       East       0.036069
2      North       0.038026
3  Northeast       0.040617
4      South       0.040617
5       West       0.037337


Summary of Recovery Rates Across Branches (Relationship Managers):


Relationship_Manager_ID  Recovery_Rate
0                  RM0001       0.026855
1                  RM0002       0.068475
2                  RM0003       0.041400
3                  RM0004       0.079029
4                  RM0005       0.024527


Observations on Recovery Rates Across Regions and Branches:
Regional recovery rates show some variation, with 'Central' having the highest average recovery rate and 'East' the lowest among the displayed regions.
Recovery rates also vary significantly across different Relationship Managers, indicating potential differences in collection effectiveness at the branch level (as Relationship Managers are proxies for branches).
